In [1]:
import pandas as pd
import numpy as np

In [2]:
authors = pd.read_csv('authors.csv')
books = pd.read_csv('Books.csv')
publishers = pd.read_csv('Publishers.csv')
users = pd.read_csv('Users.csv')

In [3]:
authors.head(10)

,first name,last name,dob
0,Danielle,Steele,3/2/1967
1,Donald,Knuth,1/10/1938
2,Brian,Kernigan,2/28/1951
3,Herbert,Schild,2/28/1951
4,Larry,Wall,9/27/1957


In [4]:
books = pd.read_csv('Books.csv')
# There was an issue in the '.' in the column names so renaming them
books = books.rename(columns={'publisher.1':'publisher1','subjects.1': 'subjects1', 'notesuser.1': 'notesuer1','notebody.1': 'notebody1'})

In [5]:
books.head()

,title,author,isbn,publisher,publisher1,available,pages,summary,subjects,subjects1,notesuser,notebody,notesuer1,notebody1,language
0,Southern Lights,Danielle Steel,303041974,Random House,Penguin Publishers,Y,2042,Danielle Steel sweeps us from a Manhattan cour...,Fiction,Romance,jason,Nice book. Must Read,jen,"Nice book. I have been traveling a lot, so I l...",English
1,Concrete Mathematics,Donald Knuth,0-203-03803-1,Addison-Wesley,NaN,Y,3524,Concrete Mathematics: A Foundation for Compute...,Mathematics,Algebra,NaN,NaN,NaN,NaN,English


In [6]:
publishers.head(10)

,name,date,street,city,zip,state,country
0,Random House,4/23/2002,1475 Broadway,New York,10019,New York,US
1,Random House,4/23/2002,375 Hudson Street,New York,10014,New York,US
2,Penguin Publishers,1/19/1998,140 Broadway,New York,10013,New York,US
3,Addison-Wesley,3/1/1994,75 Arlington Street,Boston,2116,Massachusetts,US


In [7]:
users.head(10)

,username,password,active,street,city,zip,state,country,dateOfCreation
0,ashley,ashley,Y,170 Commonwealth Avenue,Boston,02140,Massachusetts,US,4/20/2014
1,ashley,ashley,Y,10 Park Avenue,Boston,02140,Massachusetts,US,4/20/2014
2,jason,jason,Y,70 Peterbourough Street,Boston,02145,Massachusetts,US,9/20/2015
3,jason,jason,Y,2 Downtown,Boston,02143,Massachusetts,US,9/20/2015
4,admin,admin,Y,10 Geng Road,Palo Alto,94303,California,US,1/4/2014
5,jen,jen,N,Geng Road,Palo Alto,94303,California,US,11/25/2013
6,rose,rosie,60 Park Avenue,Boston,02140,Massachusetts,US,4/20/2015,NaN


In [8]:
import pymongo
# Create a MongoClient object
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# Create a database called "mydatabase":
mydb = myclient["mondoDatabase"]

In [9]:
data = authors.to_dict(orient='records')  # needed before inserting df into mondo

In [10]:
# Insert the authors df into the mongo db
mydb.myCollection.insert_many(data)

In [11]:
# Print the list of databases (mongo db isn't actually created till it has atleast one record inserted into it)
print(myclient.list_database_names())

['admin', 'config', 'local', 'mondoDatabase', 'pymongo_test', 'test']


In [12]:
# Add the rest of the dataframes into the mongo db

users = users.to_dict(orient='records')  # needed before inserting df into mondo
publishers = publishers.to_dict(orient='records')
books = books.to_dict(orient='records')

mydb.myCollection.insert_many(users)
mydb.myCollection.insert_many(publishers)
mydb.myCollection.insert_many(books)

In [13]:
#2. Write Mongo expressions to answer the following queries: (50 points) 
#+ 2.0 How many books are in the collection?
#There are 2 books in the collection
b = mydb.myCollection.find().distinct("title")
len(b)

2

In [66]:
# + 2.1 Retrieve all information on all of the books (All data associated with the books the book data, the author data, the publisher's data).
cursor = mydb.myCollection # choosing the collection you need

for document in cursor.find():
    print (document)

{'_id': ObjectId('5c37ec7c8f455e12a0864341'), 'dob': '3/2/1967', 'last name ': 'Steele', 'first name': 'Danielle'}
{'_id': ObjectId('5c37ec7c8f455e12a0864342'), 'dob': '1/10/1938', 'last name ': 'Knuth', 'first name': 'Donald'}
{'_id': ObjectId('5c37ec7c8f455e12a0864343'), 'dob': '2/28/1951', 'last name ': 'Kernigan', 'first name': 'Brian'}
{'_id': ObjectId('5c37ec7c8f455e12a0864344'), 'dob': '2/28/1951', 'last name ': 'Schild', 'first name': 'Herbert'}
{'_id': ObjectId('5c37ec7c8f455e12a0864345'), 'dob': '9/27/1957', 'last name ': 'Wall', 'first name': 'Larry'}
{'country': 'US', 'password': 'ashley', 'zip': '02140', 'street': '170 Commonwealth Avenue', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864346'), 'username': 'ashley', 'dateOfCreation': '4/20/2014', 'state': 'Massachusetts'}
{'country': 'US', 'password': 'ashley', 'zip': '02140', 'street': '10 Park Avenue', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864347'), 'username': 'ashley

In [71]:
#+ 2.2 Retrieve all information on the books where the author =  'Danielle Steele'
for document in cursor.find({'author': 'Danielle Steel'}):
    print (document)

{'notebody1': 'Nice book. I have been traveling a lot, so I listened to the audio of this book. I really enjoyed it.', 'available': 'Y', 'pages': 2042, 'subjects1': 'Romance', '_id': ObjectId('5c37f5ff8f455e12a0864367'), 'notebody': 'Nice book. Must Read', 'author': 'Danielle Steel', 'subjects': 'Fiction', 'notesuser': 'jason', 'summary': 'Danielle Steel sweeps us from a Manhattan courtroom to the Deep South in her powerful new novel. It is a behind-closed-doors look into the heart of a family and a tale of crime and punishment.', 'notesuer1': 'jen', 'language': 'English', 'publisher': 'Random House', 'publisher1': 'Penguin Publishers', 'isbn': '303041974', 'title': 'Southern Lights'}


In [74]:
#+ 2.3 Retrieve all information on the users where the user id creation is  > 15 DEC 2014 and the city = 'Boston' .
for document in cursor.find({"city":"Boston"}):
    print (document)

{'country': 'US', 'password': 'ashley', 'zip': '02140', 'street': '170 Commonwealth Avenue', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864346'), 'username': 'ashley', 'dateOfCreation': '4/20/2014', 'state': 'Massachusetts'}
{'country': 'US', 'password': 'ashley', 'zip': '02140', 'street': '10 Park Avenue', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864347'), 'username': 'ashley', 'dateOfCreation': '4/20/2014', 'state': 'Massachusetts'}
{'country': 'US', 'password': 'jason', 'zip': '02145', 'street': '70 Peterbourough Street', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864348'), 'username': 'jason', 'dateOfCreation': '9/20/2015', 'state': 'Massachusetts'}
{'country': 'US', 'password': 'jason', 'zip': '02143', 'street': '2 Downtown', 'city': 'Boston', 'active': 'Y', '_id': ObjectId('5c37f0668f455e12a0864349'), 'username': 'jason', 'dateOfCreation': '9/20/2015', 'state': 'Massachusetts'}
{'country': 'US', 'city': 